In [1]:
# Use first block to import generic non-LUSID packages
import io
import os
import pandas as pd
import numpy as np
import json
import pytz
from IPython.core.display import HTML
from datetime import datetime

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)


scope = "srs-example"

In [2]:
df= pd.read_csv('Cash_Example.csv')

In [3]:
srs_api = api_factory.build(lu.StructuredResultDataApi)
instruments_api = api_factory.build(lu.InstrumentsApi)

In [4]:
### Not required for retrival

srs_data_map = lm.DataMapping(
    data_definitions=[

        # composite key         
        lm.DataDefinition(address="UnitResult/Portfolio/Portfolio_Code", name="Portfolio", data_type="string", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/Portfolio/Currency", name="Currency", data_type="string", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/Portfolio/Class", name="Class", data_type="string", key_type="Leaf"),
        lm.DataDefinition(address="UnitResult/Instrument/default/LusidInstrumentId", name="Identifier", data_type="string", key_type="Leaf"),

        # holding values         
        lm.DataDefinition(address="UnitResult/Units", name="Units", data_type="decimal", key_type="Leaf"),
    ]
)

srs_data_map_key = lm.DataMapKey(version="0.006", code="cash-flow-map")

# display(srs_data_map)

try:    
    srs_api.create_data_map(
        scope=scope, 
        request_body={
            "market-valuation-map": lm.CreateDataMapRequest(
                id=srs_data_map_key,
                data=srs_data_map
            )
        }
    )
except lu.ApiException as e:
    display(json.loads(e.body))

{'name': 'StandardResourceConflict',
 'errorDetails': [],
 'code': 461,
 'type': 'https://docs.lusid.com/#section/Error-Codes/461',
 'title': 'Could not validate command. Reason given: DataMap exists',
 'status': 404,
 'detail': 'Could not validate command. Reason given: DataMap exists',
 'instance': 'https://omar.lusid.com/app/insights/logs/0HMC1QPUMOTQK:0000000B',
 'extensions': {}}

## Upload all records effective 9am

In [5]:
effective_date = datetime(2021, 9, 20,9, tzinfo=pytz.utc)

srs_data_id = lm.StructuredResultDataId(
        source = "Client",
        code = "CashData",
        effective_at = effective_date,
        result_type = "cashflow"
    )

srs_df = df
    
s = io.StringIO()
srs_df.to_csv(s)
    
srs_data = lm.StructuredResultData(
        document_format="Csv",
        version="0.1.1",
        name="Market valuations",
        data_map_key=srs_data_map_key,
        document=s.getvalue()        
)
    
srs_api.upsert_structured_result_data(
    scope=scope, 
    request_body={ 
        "data": lm.UpsertStructuredResultDataRequest(
            id=srs_data_id, 
                data=srs_data
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://omar.lusid.com/app/insights/logs/0HMC1QPTTKVHU:00000016',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'data': datetime.datetime(2021, 9, 27, 17, 12, 11, 811455, tzinfo=tzlocal())}}

## See result ouput

In [6]:
key= 'key'

values = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            key: srs_data_id
    }
)
    
s = io.StringIO(values.values[key].document)
values_df = pd.read_csv(s)
    
display(values_df)

,Unnamed: 0,Identifier,Portfolio_Code,Currency,Class,Units
0,0,AV_CASH,port_1,USD,Available,"10,000"
1,1,COM_CASH,port_1,USD,Committed Cash,"15,000"
2,2,SET_CASH,port_1,USD,CashAtBroker,1000


## Update record for midday - Reuplaod entire data set

In [7]:
df_2 = df
df_2.loc[0,'Units'] = 20000

In [8]:
effective_date = datetime(2021, 9, 20,12, tzinfo=pytz.utc)

srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="CashData",
        effective_at=effective_date,
        result_type = "cashflow"
    )

srs_df = df_2
    
s = io.StringIO()
srs_df.to_csv(s)
    
srs_data = lm.StructuredResultData(
        document_format="Csv",
        version="0.1.1",
        name="Market valuations",
        data_map_key=srs_data_map_key,
        document=s.getvalue()        
)
    
srs_api.upsert_structured_result_data(
    scope=scope, 
    request_body={ 
        "data": lm.UpsertStructuredResultDataRequest(
            id=srs_data_id, 
                data=srs_data
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://omar.lusid.com/app/insights/logs/0HMC1QPTTKVHU:00000018',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'data': datetime.datetime(2021, 9, 27, 17, 12, 12, 85185, tzinfo=tzlocal())}}

In [9]:
effective_date = datetime(2021, 9, 20,12, tzinfo=pytz.utc)

srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="CashData",
        effective_at=effective_date,
        result_type = "cashflow"
    )


key= 'key'

values = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            key: srs_data_id
    }
)
    
s = io.StringIO(values.values[key].document)
values_df = pd.read_csv(s)
    
display(values_df)

,Unnamed: 0,Identifier,Portfolio_Code,Currency,Class,Units
0,0,AV_CASH,port_1,USD,Available,20000
1,1,COM_CASH,port_1,USD,Committed Cash,"15,000"
2,2,SET_CASH,port_1,USD,CashAtBroker,1000


In [10]:
values

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://omar.lusid.com/app/insights/logs/0HMC1QQ5MR4FO:00000005',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'key': {'data_map_key': {'code': 'cash-flow-map',
                                     'version': '0.006'},
                    'document': ',Identifier,Portfolio_Code,Currency,Class,Units\n'
                                '0,AV_CASH,port_1,USD,Available,20000\n'
                                '1,COM_CASH,port_1,USD,Committed '
                                'Cash,"15,000"\n'
                                '2,SET_CASH,port_1,USD,CashAtBroker,1000\n',
                    'document_format': 'Csv',
                    'name': 'Market valuations',
                    'version': '0.1.1'}}}

# Add records for T+1

In [11]:
## Remove one line from df

df_t2=df.drop(1,axis=0)

In [13]:
effective_date = datetime(2021, 9, 21, tzinfo=pytz.utc)

srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="CashData",
        effective_at=effective_date,
        result_type = "cashflow"
    )

srs_df = df_t2
    
s = io.StringIO()
srs_df.to_csv(s)
    
srs_data = lm.StructuredResultData(
        document_format="Csv",
        version="0.1.1",
        name="Market valuations",
        data_map_key=srs_data_map_key,
        document=s.getvalue()        
)
    
srs_api.upsert_structured_result_data(
    scope=scope, 
    request_body={ 
        "data": lm.UpsertStructuredResultDataRequest(
            id=srs_data_id, 
                data=srs_data
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://omar.lusid.com/app/insights/logs/0HMC1QPUMOTQK:0000000C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'data': datetime.datetime(2021, 9, 27, 17, 12, 12, 501448, tzinfo=tzlocal())}}

## Querying data T+1

In [14]:
effective_date = datetime(2021, 9, 21, tzinfo=pytz.utc)

srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="CashData",
        effective_at=effective_date,
        result_type = "cashflow"
    )


key= 'key'

values = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            key: srs_data_id
    }
)
    
s = io.StringIO(values.values[key].document)
values_df = pd.read_csv(s)
    
display(values_df)

,Unnamed: 0,Identifier,Portfolio_Code,Currency,Class,Units
0,0,AV_CASH,port_1,USD,Available,20000
1,2,SET_CASH,port_1,USD,CashAtBroker,1000


## Querying data for T

In [15]:
effective_date = datetime(2021, 9, 20,9, tzinfo=pytz.utc)

srs_data_id = lm.StructuredResultDataId(
        source="Client",
        code="CashData",
        effective_at=effective_date,
        result_type = "cashflow"
    )


key= 'key'

values = srs_api.get_structured_result_data(
        scope=scope, 
        request_body={
            key: srs_data_id
    }
)
    
s = io.StringIO(values.values[key].document)
values_df = pd.read_csv(s)
    
display(values_df)

,Unnamed: 0,Identifier,Portfolio_Code,Currency,Class,Units
0,0,AV_CASH,port_1,USD,Available,"10,000"
1,1,COM_CASH,port_1,USD,Committed Cash,"15,000"
2,2,SET_CASH,port_1,USD,CashAtBroker,1000
